## **Exercício 3**

### Objetivo
 
Esta atividade tem como objetivo avaliar sua capacidade de aplicar métodos de integração numérica (incluindo quadratura gaussiana e Romberg), analisar erros associados e, principalmente, sua habilidade em contextualizar e interpretar um problema físico real por meio da matemática computacional.

Você deverá:

- Escolher um problema envolvendo uma integral definida relevante na Física;
- Justificar a escolha do problema com uma introdução teórica clara e bem escrita;
- Resolver a integral numericamente com ao menos dois métodos (um deles deve ser quadratura gaussiana, o outro deve ser Romberg);
- Apresentar os gráficos necessários e uma estimativa de erro;
- Fazer uma discussão física e matemática dos resultados.

### Etapas da atividade

a. Escolha e contextualização do problema físico

Escolha uma função física integrável definida no intervalo $[a,b]$, com $a \geq 0$, e que possa representar alguma grandeza relevante na Física (exemplos possíveis incluem):
    - Energia média de uma partícula em um sistema estatístico;
    - Probabilidade acumulada de uma função de onda;
    - Densidade de corrente, fluxo, intensidade de campo;
    - Capacidade térmica, densidade de estados, função de resposta espectral.

**A função deve ser não trivial (comportamento exponencial, oscilatório, singularidade removível ou decaimento polinomial). Evite funções polinomiais simples ou integrais que possam ser resolvidas facilmente de forma exata.**

Você deverá escrever um texto introdutório com:
    - O contexto físico do problema;
    - A origem da integral escolhida (com unidades e significado físico);
    - As possíveis dificuldades numéricas na avaliação dessa integral.

b. Cálculo da integral com dois métodos

Implemente os dois métodos de integração:
    - Quadratura Gaussiana (obrigatória);
    - Romberg (obrigatória).

c. Estimativa de erro

- Para cada método, estime o erro associado (por comparação entre refinamentos sucessivos);
- Se possível, compare com o valor teórico (analítico) da integral, caso exista;
- Comente qual método se mostrou mais eficiente para o seu caso.

d. Discussão dos resultados

- Apresente ao menos um gráfico da função integranda e, se aplicável, da função acumulada;
- Interprete o resultado final fisicamente;
- Comente sobre a estabilidade numérica dos métodos;
- Justifique suas escolhas e o comportamento observado.

Parte reflexiva

- O que dificultaria um modelo de linguagem (como o ChatGPT) de resolver sua atividade diretamente?
- Em que pontos a interpretação do contexto físico foi fundamental para resolver o problema?
- Houve diferença significativa entre os métodos? Qual se mostrou mais adequado ao seu caso?